In [3]:
# Let's do our usual data science imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import importlib
import seaborn as sns
import os
import subprocess
sns.set(style="darkgrid")

# Define a number of constants
PATH_TO_LOG = "storage/logs/"
PATH_TO_PLOTS = "storage/plots/"
    
EVALUATION_SCHAFFERS = "SchaffersEvaluation"
EVALUATION_KATSUURA = "KatsuuraEvaluation"
EVALUATION_BCF = "BentCigarFunction"
EVALUATION_SPHERE = "SphereEvaluation"

PARAMETER_POPSIZE = "popsize"
PARAMETER_MOMENTUM = "momentum"
PARAMETER_MUTATION = "mutation"
PARAMETER_SIGMA = "sigma"

In [4]:
# Helper functions
def makeDirStructure(path):
    if not os.path.exists(os.path.dirname(path)):
        try:
            os.makedirs(os.path.dirname(path))
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

In [5]:
# Define the Setup class
class Setup:
    """A setup for a defined parameter and evaluation, with given parameter values.

    Params:
        - param_name (str): Parameter to define
        - evaluation (str): Name of evaluation function to optimise.
        - parameter_values (list): List of values for the associated parameter
    """
    def __init__(self, evaluation, param_name = "", parameter_values = []):
        self.param_name = param_name
        self.evaluation = evaluation
        self.parameter_values = parameter_values
        self.parameter_scores = []

In [6]:
# Define an Evaluation class
class Evaluation:
    def __init__(self, eval_name, parameters):
        self.name = eval_name
        self.parameters = parameters

In [36]:
# Define the Simulator class
class Simulator:
    """A simulator for a givenn number of setups, and runs to test on each parameter value of a setup.

    Params:
        - setups(list): A list of Setup to test on
        - runs(int): Number of runs per setup
    """
    
    def __init__(self, setups, runs=10):
        self.setups = setups
        self.runs = runs

    def run(self):
        for setup in self.setups:
            self.runEvalsForSetup(setup)

    def constructParams(self, setup, param):
        """Sets up default and preconfigured parameters."""
        setup.evaluation.parameters[setup.param_name] = param

    def runEvalsForSetup(self, setup):
        """Runs for the current Setup:setup of parameters to test. """
        for param in setup.parameter_values:
            self.runEval(setup, param)

    def runEval(self, setup, param):
        """Runs the EA on the igiven eval"""
        scores = 0
        print(scores)
        
        for i in range(self.runs):
            subprocess.run("bash run.sh clean", shell=True)
            subprocess.run("bash run.sh compile", shell=True)
            cmd = "bash run.sh %s %d %f %d" % (setup.evaluation.name, setup.evaluation.parameters[PARAMETER_POPSIZE], setup.evaluation.parameters[PARAMETER_SIGMA], setup.evaluation.parameters[PARAMETER_MOMENTUM])
            p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
            output = p.stdout.readlines()[0]
            self.storeDataToFiles(output, setup, param, i)
            
    def storeDataToFiles(self, output, setup, param, run_nr):
        """Stores output to corresponding setup location."""
        path = PATH_TO_LOG + "{}/{}/{}-val-{}-run-{}.txt".format(setup.evaluation.name, setup.param_name, setup.param_name, param, run_nr)
        makeDirStructure(path)
        
        file = open(path, "w+")
        file.write(str(output.decode()))
        file.close()
    
    def readDataForSetup(self, setup):
        """Reads prior-run setup from filesystem."""
        end_df = pd.DataFrame()

        for index, val_metric in enumerate(setup.parameter_values):
            df = pd.DataFrame()
            
            for i in range(self.runs):
                path = PATH_TO_LOG + "{}/{}/{}-val-{}-run-{}.txt".format(setup.evaluation.name, setup.param_name, setup.param_name, val_metric, i)

                with open(path) as file:
                    output = file.readlines()

            output_reals = eval(output[0])
            df[index] = output_reals

            end_df[index] = df.mean(axis='columns')
            
        return end_df
    
    def generatePlotForSetup(self, setup, start_index=0, threshold=10.0, output_data=pd.DataFrame()):
    def generatePlotForSetup(self, setup, start_index=0, threshold=10.0):
        path = PATH_TO_PLOTS + "{}/{}.png".format(setup.evaluation,setup.param_name)
        """Generates a plot, assuming your setup has been run and stored in storage."""
        path = PATH_TO_PLOTS + "{}/{}-parameters.png".format(setup.evaluation.name, setup.param_name)
        
        if output_data.size < 1:
            output_data = self.readDataForSetup(setup)
        
        sns.relplot(hue="region", kind="line", data=output_data[output_data < threshold].iloc[start_index:])
        makeDirStructure(path)
        plt.savefig(path)
        

In [37]:
# Constructing the default evaluation objects (no need to touch this!)
Katsuura_eval = Evaluation(
    EVALUATION_KATSUURA,
    {
        PARAMETER_POPSIZE: 100,
        PARAMETER_MUTATION: 0.5,
        PARAMETER_SIGMA: 0.1,
        PARAMETER_MOMENTUM: 0
    }
)

Schaffers_eval = Evaluation(
    EVALUATION_SCHAFFERS,
    {
        PARAMETER_POPSIZE: 100,
        PARAMETER_MUTATION: 0.6,
        PARAMETER_SIGMA: 1,
        PARAMETER_MOMENTUM: 0
    }
)

Sphere_eval = Evaluation(
    EVALUATION_SPHERE,
    {
        PARAMETER_POPSIZE: 100,
        PARAMETER_MUTATION: 0.25,
        PARAMETER_SIGMA: 1,
        PARAMETER_MOMENTUM: 0
    }
)

BCF_eval = Evaluation(
    EVALUATION_BCF,
    {
        PARAMETER_POPSIZE: 100,
        PARAMETER_MUTATION: 0.25,
        PARAMETER_SIGMA: 1,
        PARAMETER_MOMENTUM: 0
    }
)

In [38]:
# Initialize a number of setups
katsuura_popsize_setup = Setup(
    Katsuura_eval,
    PARAMETER_POPSIZE, 
    np.linspace(50, 500, 50)
)

schaffers_popsize_setup = Setup(
    Schaffers_eval,
    PARAMETER_POPSIZE,
    [50, 100]
)

schaffers_momentum_setup = Setup(
    Schaffers_eval,
    PARAMETER_MOMENTUM,
    np.arange(0, 0.6, 0.1)
)

bcf_popsize_setup = Setup(
    BCF_eval,
    PARAMETER_POPSIZE,
    [50, 100]
)

bcf_momentum_setup = Setup(
    BCF_eval,
    PARAMETER_MOMENTUM,
    np.arange(0, 0.6, 0.1)
)

In [39]:
# Definining the setups
setups = [bcf_momentum_setup]
mainSimulator = Simulator(setups)

In [22]:
data = mainSimulator.readDataForSetup(setups[0])

In [40]:
mainSimulator.generatePlotForSetup(setups[0], output_data=data)

## Generating New Data from EA

## New heading

In [ ]:
# Only run this if you want to make new data

mainSimulator = Simulator(setups)
mainSimulator.run()

0
0


## Playing around

In [ ]:
plt.show()

In [ ]:
# first_entry = schaffers_popsize_results[0].iloc[9000:15000]
# second_entry = schaffers_popsize_results[1].iloc[9000:15000]
# third_entry = schaffers_popsize_results[2].iloc[9000:15000]
# fourth_entry = schaffers_popsize_results[3].iloc[9000:15000]
# fifth_entry = schaffers_popsize_results[4].iloc[9000:15000]
# first_entry[first_entry < 9.999].plot()
# second_entry[second_entry < 9.999].plot()
# third_entry[third_entry < 9.999].plot()
# fourth_entry[fourth_entry < 9.999].plot()
# fifth_entry[fifth_entry < 9.999].plot()
# plt.show()

# sns.relplot(hue="coherence", style="choice",
#             kind="line", data=schaffers_popsize_results[schaffers_popsize_results < 9.999]);

In [ ]:
# from mpl_toolkits.mplot3d import Axes3D
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# plt.show()